In [1]:
%pip install pygame
import numpy as np
import math
import pygame
import sys
import random
import copy
import os

     |████████████████████████████████| 21.8 MB 1.1 MB/s 
pygame 2.1.2 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
FPS = 60
BLACK = (0, 0, 0)
WITHE = (255, 255, 255)
YELLOW = (255, 255, 0)
VISION = 100

In [3]:
class AI(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.genom = [np.random.rand(4, 5), np.random.rand(5, 2)]
        self.x = random.randint(0, 1000)
        self.y = random.randint(0, 1000)
        self.xV = 0
        self.yV = 0
        self.food = random.randint(0, 10)
        self.damage = random.randint(0, 10)
        self.defend = random.randint(0, 10)
        self.energy = 100
        self.image = pygame.transform.scale(pygame.image.load('individ.png').convert_alpha(), (20, 20))
        self.rect = self.image.get_rect(center=(self.x, self.y))
    
    def f(self, x):
        return (math.e ** x - math.e ** (-x)) / (math.e ** x + math.e ** (-x))
    
    def calculate(self, inputValue):
        sig = np.vectorize(self.f)
        hiddenValue = np.matrix(sig(inputValue * self.genom[0]))
        outputValue = sig(hiddenValue * self.genom[1])
        return np.array(outputValue)
        
    def do(self, inputValue):
        outputValue = self.calculate(inputValue)
        self.xV += np.sign(outputValue[0][0]) if self.xV <= 10 else 0
        self.yV += np.sign(outputValue[0][1]) if self.yV <= 10 else 0
        self.x += self.xV
        self.y += self.yV
        self.rect.x = self.x
        self.rect.y = self.y
        self.energy -= 1
        
    def eat(self):
        self.energy += 50
    
    def copy(self):
        newIndivid = AI()
        newIndivid.x = self.x + 10
        newIndivid.y = self.y + 10
        newIndivid.genom = [list(self.genom[0]), list(self.genom[1])]
        rand = random.randint(0, 100)
        if rand < 5:
            newIndivid.genom[0][random.randint(0, 3)][random.randint(0, 4)] = random.random()
        elif rand > 5 and rand < 10:
            newIndivid.genom[1][random.randint(0, 4)][random.randint(0, 1)] = random.random()
        newIndivid.genom = [np.array(newIndivid.genom[0]), np.array(newIndivid.genom[1])]
        newIndivid.energy = self.energy // 2
        self.energy //= 2
        return newIndivid

In [4]:
class Food(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.x = np.random.randint(0, 1000)
        self.y = np.random.randint(0, 1000)
        self.image = pygame.transform.scale(pygame.image.load('food.jpg').convert_alpha(), (10, 10))
        self.rect = self.image.get_rect(center=(self.x, self.y))

In [5]:
class AI(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.genom = [np.random.rand(4, 5), np.random.rand(5, 2)]
        self.x = random.randint(0, 1000)
        self.y = random.randint(0, 1000)
        self.xV = 0
        self.yV = 0
        self.food = random.randint(0, 10)
        self.damage = random.randint(0, 10)
        self.defend = random.randint(0, 10)
        self.energy = 100
        self.image = pygame.transform.scale(pygame.image.load('individ.png').convert_alpha(), (20, 20))
        self.rect = self.image.get_rect(center=(self.x, self.y))
    
    def f(self, x):
        return (math.e ** x - math.e ** (-x)) / (math.e ** x + math.e ** (-x))
    
    def calculate(self, inputValue):
        sig = np.vectorize(self.f)
        hiddenValue = np.matrix(sig(inputValue * self.genom[0]))
        outputValue = sig(hiddenValue * self.genom[1])
        #print(outputValue)
        return np.array(outputValue)
        
    def do(self, inputValue):
        outputValue = self.calculate(inputValue)
        self.xV += np.sign(outputValue[0][0]) if self.xV <= 10 else 0
        self.yV += np.sign(outputValue[0][1]) if self.yV <= 10 else 0
        self.x += self.xV
        self.y += self.yV
        self.rect.x = self.x
        self.rect.y = self.y
        self.energy -= 1
        
    def eat(self):
        self.energy += 50
    
    def copy(self):
        newIndivid = AI()
        newIndivid.x = self.x + 10
        newIndivid.y = self.y + 10
        newIndivid.genom = [list(self.genom[0]), list(self.genom[1])]
        rand = random.randint(0, 100)
        if rand < 5:
            newIndivid.genom[0][random.randint(0, 3)][random.randint(0, 4)] = random.random()
        elif rand > 5 and rand < 10:
            newIndivid.genom[1][random.randint(0, 4)][random.randint(0, 1)] = random.random()
        newIndivid.genom = [np.array(newIndivid.genom[0]), np.array(newIndivid.genom[1])]
        newIndivid.energy = self.energy // 2
        self.energy //= 2
        return newIndivid

In [6]:
class Food(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.x = np.random.randint(0, 1000)
        self.y = np.random.randint(0, 1000)
        self.image = pygame.transform.scale(pygame.image.load('food.jpg').convert_alpha(), (10, 10))
        self.rect = self.image.get_rect(center=(self.x, self.y))

In [7]:
class Manager:
    def __init__(self):
        self.individs = [AI() for i in range(25)]
        self.foods = [Food() for i in range(200)]
        
    def dist(self, a, b):
        return ((a.x - b.x) ** 2 + (a.y - b.y) ** 2) ** 0.5
    
    def data(self, objects):
        vis = [[np.array([j.x, j.y]) for j in objects if self.dist(i, j) <= VISION and i != j] for i in self.individs]
        v = [sum(i) // len(i) if len(i) != 0 else np.array([0, 0]) for i in vis]
        return v
    
    def eat(self, per):
        [[self.foods.remove(i), j[0].eat()] for i, j in per.items()]
        
    def step(self):
        inputEats = self.data(self.foods)
        inputIndivids = self.data(self.individs)
        [ai.do(np.matrix([(eat[0] - ai.x) / VISION, (eat[1] - ai.y) / VISION, (individ[0] - ai.x) / VISION, (individ[1] - ai.y) / VISION])) for ai, eat, individ in zip(self.individs, inputEats, inputIndivids)]
        self.eat(pygame.sprite.groupcollide(self.foods, self.individs, False, False, pygame.sprite.collide_mask))
        [self.individs.remove(i) for i in self.individs if i.energy <= 0]
        [self.individs.append(i.copy()) for i in self.individs if i.energy >= 100]
        self.foods.extend([Food() for i in range(1)])
        return self.individs, self.foods

In [8]:
def engEvent(): 
    for i in pygame.event.get(): 
        if i.type == pygame.QUIT: 
            pygame.quit() 
            sys.exit() 

In [9]:
pygame.init()
os.environ['SDL_VIDEODRIVER'] = 'dummy'
screen = pygame.display.set_mode((1000, 1000), pygame.HWSURFACE|pygame.DOUBLEBUF) 
pygame.display.set_caption("My Game") 
clock = pygame.time.Clock() 

manager = Manager()
 
t = 0 
while 1: 
    engEvent() 
    clock.tick(60) 
    individs, foods = manager.step()
     
    [screen.blit(i.image, i.rect) for i in individs]
    
    [screen.blit(i.image, i.rect) for i in foods]
    
    if len(individs) <= 0:
        manager = Manager()
    
     
    t += 0.1 
    pygame.display.flip() 
    screen.fill(WITHE)

FileNotFoundError: ignored